## Foursquare Coursera Capstone: Battle of the Neighbourhoods

This Notebook will be used for the Capstone Project

In [1]:
import numpy as np ### IMPORTING ALL RELEVANT MODULES
import pandas as pd
import requests
import random
import folium
import geopy
import json
from clusteval import clusteval

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Hello Capstone Project Course!')

print ('All modules loaded')


Hello Capstone Project Course!
All modules loaded


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  ###READING THE TABLE FORM THE URL INTO PD DATAFRAME
df = pd.read_html(url)[0] ###AMAZING!!!!!!
df.head(25)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
df['Postal Code'].value_counts()

M2A    1
M7W    1
M6Z    1
M8P    1
M8W    1
      ..
M4B    1
M5Y    1
M2C    1
M3L    1
M4M    1
Name: Postal Code, Length: 180, dtype: int64

In [4]:
df = df[df.Borough != 'Not assigned']  ##DROPPING BOROUGHS THAT ARE NOT ASSIGNED

In [5]:
df.shape

(103, 3)

I have done part one of week 3 at this point. The dataframe above is the result. Each cell (where neccessary) is commented 

#### Adding coordinate data to borough dataframe

In [6]:
df.insert(3, "Latitude", 0)  ###CREATES TWO NEW COLUMNS TO BE POPULATED 
df.insert(4, "Longitude", 0)

In [7]:
df.reset_index(drop=True)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",0,0
99,M4Y,Downtown Toronto,Church and Wellesley,0,0
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",0,0
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0,0


In [8]:
pd.set_option("display.precision", 13)
df_gps = pd.read_csv('Geospatial_Coordinates.csv')
df_gps

,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.7731360,-79.2394761
...,...,...,...
98,M9N,43.7068760,-79.5181884
99,M9P,43.6963190,-79.5322424
100,M9R,43.6889054,-79.5547244
101,M9V,43.7394164,-79.5884369


In [9]:
df = pd.merge(df, df_gps, on="Postal Code")  ##THIS IS AMAZING. IT MERGES THE TWO DATAFRAMES BASED ON COMMON COLUMN VALUES OF EACH DATAFRAME. 
##THE COORDINATES OF EACH POSTAL CODE ARE NOW MERGED INTO THE INITIAL DATAFRAME WITH THE BOROUGHS AND NEIGHBOURHOODS, AND THE COORDS MATCH EACH POSTAL CODE 
## BECAUSE WE HAVE TEH OTEHR DATAFRAME MAPPING EACH POSTAL CODE WITH ITS RESPECTIVE COORDINATES
df.head(25) 

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,0,0,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,0,0,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0,43.7185180,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0,43.6623015,-79.3894938
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",0,0,43.6678556,-79.5322424
6,M1B,Scarborough,"Malvern, Rouge",0,0,43.8066863,-79.1943534
7,M3B,North York,Don Mills,0,0,43.7459058,-79.3521880
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0,0,43.7063972,-79.3099370
9,M5B,Downtown Toronto,"Garden District, Ryerson",0,0,43.6571618,-79.3789371


In [10]:
df = df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude_y', 'Longitude_y']]
df = df.rename(columns={'Latitude_y': 'Latitude', 'Longitude_y': 'Longitude'})

In [11]:
df ### CLEAN DATAFRAME WITH EVERYTHING AS ASKED

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185180,-79.4647633
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6536536,-79.5069436
99,M4Y,Downtown Toronto,Church and Wellesley,43.6658599,-79.3831599
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6627439,-79.3215580
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6362579,-79.4985091


In [12]:
latitude, longitude = 43.6532, -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
##Defining Foursquare creds and other search parameters
neighborhood_latitude = df['Latitude'][0]  ## Can either loop through index or thruogh each elemnt, for i in range(len(df['Latitude'])): df['Latitude'][i]
neighborhood_longitude = df['Longitude'][0]## or for i in df['Latitude']

LIMIT = 100
radius = 750
CLIENT_ID = 'LXLE51IMWGXGMXFAWGALEWSYPTUHPOHRMIMF0KSNRO4S2V34'
CLIENT_SECRET = 'HIWNF4A0MWFLVTYRY1WQNGDFLICRZL1EWXRAQKGCHTXRLOA4'
VERSION = '20180604'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=LXLE51IMWGXGMXFAWGALEWSYPTUHPOHRMIMF0KSNRO4S2V34&client_secret=HIWNF4A0MWFLVTYRY1WQNGDFLICRZL1EWXRAQKGCHTXRLOA4&v=20180604&ll=43.7532586,-79.3296565&radius=750&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa2c5a1a6646a5e7687652c'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.760008606750006,
    'lng': -79.32032910977391},
   'sw': {'lat': 43.74650859325, 'lng': -79.33898389022609}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
print(venues)
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
print(nearby_venues)
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
print(nearby_venues)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b', 'name': 'Brookbanks Park', 'location': {'address': 'Toronto', 'lat': 43.751976046055574, 'lng': -79.33214044722958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.751976046055574, 'lng': -79.33214044722958}], 'distance': 245, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id':

,name,categories,lat,lng
0,Brookbanks Park,Park,43.7519760460556,-79.3321404472296
1,Variety Store,Food & Drink Shop,43.7519744158578,-79.3331141851602
2,DVP at York Mills,Intersection,43.7588987650483,-79.3340989154166
3,TTC Stop #09083,Bus Stop,43.7596548489696,-79.3322229677883


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(df['Neighbourhood'],
                                   df['Latitude'],
                                   df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7532586,-79.3296565,Brookbanks Park,43.7519760460556,-79.3321404472296,Park
1,Parkwoods,43.7532586,-79.3296565,Variety Store,43.7519744158578,-79.3331141851602,Food & Drink Shop
2,Parkwoods,43.7532586,-79.3296565,DVP at York Mills,43.7588987650483,-79.3340989154166,Intersection
3,Parkwoods,43.7532586,-79.3296565,TTC Stop #09083,43.7596548489696,-79.3322229677883,Bus Stop
4,Victoria Village,43.7258823,-79.3155716,Victoria Village Arena,43.7234805554551,-79.3156352092514,Hockey Arena
...,...,...,...,...,...,...,...
3685,"Mimico NW, The Queensway West, South of Bloor,...",43.6288408,-79.5209994,Tactical Products Canada,43.6268005371094,-79.5293884277344,Miscellaneous Shop
3686,"Mimico NW, The Queensway West, South of Bloor,...",43.6288408,-79.5209994,Performance Improvements,43.6352054852269,-79.5219826698303,Automotive Shop
3687,"Mimico NW, The Queensway West, South of Bloor,...",43.6288408,-79.5209994,Global Cheese,43.6323392402963,-79.5285671656385,Cheese Shop
3688,"Mimico NW, The Queensway West, South of Bloor,...",43.6288408,-79.5209994,Light Up The Floor - Your Lifestyle Studio,43.6351142455450,-79.5239282890974,Gym / Fitness Center


In [21]:
toronto_venues['Neighbourhood'].value_counts()

Central Bay Street                                   100
St. James Town                                       100
Harbourfront East, Union Station, Toronto Islands    100
Berczy Park                                          100
Church and Wellesley                                 100
                                                    ... 
Rouge Hill, Port Union, Highland Creek                 5
Cliffside, Cliffcrest, Scarborough Village West        5
Parkwoods                                              4
York Mills, Silver Hills                               2
Northwest, West Humber - Clairville                    1
Name: Neighbourhood, Length: 98, dtype: int64

In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#print(toronto_venues['Venue Category'].unique())

There are 323 uniques categories.


In [23]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]


In [24]:
tor_onehot

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3686,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3687,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3688,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.030303030303,0.0,0.0,0.0,0.0,0.030303030303,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000000000,0.0,0.0,0.0,0.0,0.000000000000,0.0,0.0


In [26]:
num_top_venues = 5   ####PRINTING OUT MOST COMMON VENUES (HIGEHST AMOUNT OF A PARTICULAR VENUE) PER NEIGHBOURHOOD (NOT THE MOST POPULAR BUT THE MOST COMMON)
###NOTICE ITS FREQ OF VENUE PER NEIHGBOURHOOD

for hood in tor_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighbourhood'] == hood].T.reset_index()
    print('temp:',temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

----Agincourt----
temp:                   index          0
0         Neighbourhood  Agincourt
1     Accessories Store          0
2     Afghan Restaurant          0
3    African Restaurant          0
4               Airport          0
..                  ...        ...
319            Wine Bar          0
320           Wine Shop          0
321         Wings Joint          0
322       Women's Store          0
323         Yoga Studio          0

[324 rows x 2 columns]
----Alderwood, Long Branch----
temp:                   index                       1
0         Neighbourhood  Alderwood, Long Branch
1     Accessories Store                       0
2     Afghan Restaurant                       0
3    African Restaurant                       0
4               Airport                       0
..                  ...                     ...
319            Wine Bar                       0
320           Wine Shop                       0
321         Wings Joint                       0
322       Women

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:   #ind(st), ind(nd) ..... where ind is 1,2,3....
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))## try: USES indicators list. when it iterates through the entire list it goes TO EXCEPT
    except: #from ind = 4 onwards: 4th, 5th, (ind)th........
        columns.append('{}th Most Common Venue'.format(ind+1))### except: ONCE THE LIST IS EXHAUSTED IT CONTINUES WITH THIS CONDITION

# create a new dataframe
tor_venues_sorted = pd.DataFrame(columns=columns)
tor_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

print(tor_venues_sorted.iloc[ind, 1:])
print((tor_grouped.shape[0]))
print(return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues))

for ind in np.arange(tor_grouped.shape[0]):
    tor_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)  #SETS ROW OF SORTED DF EUQAL TO THE OUTPUT OF THE RETUN_MOST_COMMON_VENUES FN
                                                                                        

print(tor_venues_sorted.iloc[ind, 1:]) ### ind=97 because we have iterated thru entire loop 

tor_venues_sorted.head()

1st Most Common Venue     NaN
2nd Most Common Venue     NaN
3rd Most Common Venue     NaN
4th Most Common Venue     NaN
5th Most Common Venue     NaN
6th Most Common Venue     NaN
7th Most Common Venue     NaN
8th Most Common Venue     NaN
9th Most Common Venue     NaN
10th Most Common Venue    NaN
Name: 9, dtype: object
98
['Harbor / Marina' 'Rental Car Location' 'Coffee Shop' 'Boat or Ferry'
 'Sculpture Garden' 'Airport Lounge' 'Airport Service' 'Airport Terminal'
 'Boutique' 'Park']
1st Most Common Venue             Martial Arts School
2nd Most Common Venue                            Pool
3rd Most Common Venue            Ethiopian Restaurant
4th Most Common Venue                         Dog Run
5th Most Common Venue                Doner Restaurant
6th Most Common Venue                      Donut Shop
7th Most Common Venue                     Dry Cleaner
8th Most Common Venue             Dumpling Restaurant
9th Most Common Venue     Eastern European Restaurant
10th Most Common Venue 

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Lounge,Sushi Restaurant,Sandwich Place,Supermarket,Discount Store,Breakfast Spot,Motorcycle Shop,Coffee Shop
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Gym,Gas Station,Park,Donut Shop,Sandwich Place,Pharmacy,Pub,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Coffee Shop,Mobile Phone Shop,Shopping Mall,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Sushi Restaurant,Sandwich Place
3,Bayview Village,Bank,Japanese Restaurant,Grocery Store,Shopping Mall,Café,Chinese Restaurant,Skating Rink,Intersection,Dive Bar,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Sports Club,Bank,Bakery,Bagel Shop,Butcher,Sushi Restaurant


In [29]:
tor_grouped.iloc[0, :]

Neighbourhood         Agincourt
Accessories Store             0
Afghan Restaurant             0
African Restaurant            0
Airport                       0
                        ...    
Wine Bar                      0
Wine Shop                     0
Wings Joint                   0
Women's Store                 0
Yoga Studio                   0
Name: 0, Length: 324, dtype: object

In [30]:
tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)



In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
param_grid = [{'init' : ['k-means++', 'random', 'ndarray', 'callable'], 'n_clusters' : [4,5,6,7,8,9]}]

In [33]:
km_CV = GridSearchCV(KMeans(), param_grid, cv=5, verbose=True, n_jobs=8)
km_CV.fit(tor_grouped_clustering)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed:    3.6s finished


GridSearchCV(cv=5, estimator=KMeans(), n_jobs=8,
             param_grid=[{'init': ['k-means++', 'random', 'ndarray',
                                   'callable'],
                          'n_clusters': [4, 5, 6, 7, 8, 9]}],
             verbose=True)

In [34]:
print(km_CV.best_estimator_)
print(km_CV.best_score_)
print(km_CV.best_params_)

KMeans(n_clusters=9)
-1.5531528316675722
{'init': 'k-means++', 'n_clusters': 9}


In [37]:
km = KMeans(n_clusters=7, init= 'k-means++', random_state=0)

km.fit(tor_grouped_clustering)

km.labels_[0:500] 

array([0, 1, 1, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 5, 0, 1, 3, 1, 0, 0, 1, 1, 1, 1, 0, 3, 0,
       0, 1, 0, 2, 0, 0, 0, 1, 0, 3, 0, 3, 0, 6, 0, 5, 0, 1, 3, 0, 0, 0,
       5, 6, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1,
       1, 3, 1, 2, 0, 2, 2, 3, 2, 4])

In [38]:
# tor_grouped_clustering.insert(0, 'Cluster Labels', km.labels_)

# tor_merged = df.join(tor_grouped_clustering.set_index('Neighbourhood'), on='Neighbourhood')


# # merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
# tor_merged = tor_merged.join(tor_grouped_clustering.set_index('Neighbourhood'), on='Neighbourhood')

# tor_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters=8
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], km.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters